In [2]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import json as js


def get_keywords(f_key_dict):
    dicts = {}
    for lag in languages:
        dicts[lag] = {'lemmas':[]}
    for l in open(f_key_dict):
        obj = js.loads(l)
        lag = obj['language']
        if lag in languages:
            lemmas = [item['lemma'] for item in obj['tokens']]
            lemmas = filter(lambda x: len(x) > 1, lemmas)
            if len(lemmas) == 1:
                dicts[lag][lemmas[0]] = 0
            elif len(lemmas) == 2:
                dicts[lag]["_".join(lemmas)] = 0
            elif len(lemmas) == 3:
                dicts[lag]["_".join(lemmas)] = 0
    return dicts


def create_y(f, date_range):
    df_gsr = pd.DataFrame(columns=('date', 'event', 'city'))
    lines = open(f).readlines()
    for i in range(len(lines)):
        obj = js.loads(lines[i])
        if obj['location'][0] == 'Brazil':
            df_gsr.loc[i] = [pd.to_datetime(obj['eventDate'].split("T")[0], format='%Y-%m-%d') + timedelta(days=-1),
                             1,
                             obj['location'][1]]

    df_gsr_selected = df_gsr[(df_gsr.date >= min_date) & (df_gsr.date <= max_date)]
    df_gsr_selected = df_gsr_selected[df_gsr_selected['city'] != '-']
    df_grouped = df_gsr_selected.groupby(['city', 'date']).agg({'event': lambda x: 1})

    cities = pd.unique(df_gsr_selected.city.ravel())
    new_index = pd.MultiIndex.from_product([cities, date_range], names=['city', 'date'])

    df_final = df_grouped.reindex(new_index, fill_value=0)
    return df_final

In [3]:
file_gsr = "./gsrAll.json"
languages = ['English', 'Portuguese', 'Spanish']
min_date = datetime(2014, 1, 1)
max_date = datetime(2014, 12, 31)
dates = pd.date_range(min_date, max_date)
y = create_y(file_gsr, dates)

file_keys = "./CU_Keywords.2013-01-25T15-36-29"
keywords_dict = get_keywords(file_keys)[languages[1]]
keywords = keywords_dict.keys()
col_names = keywords + ['date', 'city']
types = {key: 'int' for key in keywords}
types['date'] = 'str'
types['city'] = 'str'
X = pd.read_csv('features_Portuguese.csv', header=None, names=col_names)
X = X.groupby(['city', 'date']).agg({key: np.sum for key in keywords})

In [4]:
X.shape

(9855, 676)

In [5]:
y.shape

(9855, 1)

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
model = LogisticRegression(penalty='l1')

In [8]:
model.coef_

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

In [9]:
model.fit(X,y)

C:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
model.coef_

array([[-0.15225976,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.13223792,  0.        , -0.2400497 ,  0.        ,  0.        ,
         0.        ,  0.        , -0.09150101,  0.        ,  0.        ,
         0.        ,  0.        ,  0.07386145,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.03176854,
         0.        ,  0.        ,  0.        ,  0.        , -0.25547892,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.16843067, -0.37372135,  0.        ,  0.        ,  0.        ,
         0.17162867,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , -0.2128016 ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.3

In [11]:
len(model.coef_[model.coef_ != 0])

223

In [12]:
len(model.coef_)

1

In [13]:
len(model.coef_[model.coef_ == 0])

453

In [14]:
y.shape

(9855, 1)

In [15]:
X.shape

(9855, 676)

In [16]:
model.predict_log_proba(X)

array([[-0.1155111 , -2.21558831],
       [-0.1155111 , -2.21558831],
       [-0.1155111 , -2.21558831],
       ..., 
       [-0.13442376, -2.07321714],
       [-0.05224476, -2.97782432],
       [-0.0524967 , -2.97313855]])

In [17]:
model.predict_proba(X)

array([[ 0.89091068,  0.10908932],
       [ 0.89091068,  0.10908932],
       [ 0.89091068,  0.10908932],
       ..., 
       [ 0.87421952,  0.12578048],
       [ 0.94909654,  0.05090346],
       [ 0.94885746,  0.05114254]])

In [18]:
model.predict(X)

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [19]:
len(model.coef_[model.coef_ == 0], class_weight = 'balanced')

TypeError: len() takes no keyword arguments

In [20]:
model = LogisticRegression(penalty='l1', class_weight = 'balanced')

In [21]:
model.fit(X, y)
print(model)
# make predictions
expected = y
predicted = model.predict(X)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)


C:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


NameError: name 'metrics' is not defined

In [22]:
from sklearn import metrics

In [23]:
print(metrics.classification_report(expected, predicted))


             precision    recall  f1-score   support

        0.0       1.00      0.26      0.42      8959
        1.0       0.12      1.00      0.21       896

avg / total       0.92      0.33      0.40      9855



In [24]:
print(metrics.confusion_matrix(expected, predicted))

[[2352 6607]
 [   0  896]]


In [25]:
len(y[y.event != 0])

896

In [26]:
print(model)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)


In [27]:
model = LogisticRegression(penalty='l2', class_weight='balanced')
model.fit(X, y)
expected = y
predicted = model.predict(X)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

        0.0       1.00      0.25      0.40      8959
        1.0       0.12      1.00      0.21       896

avg / total       0.92      0.32      0.38      9855

[[2241 6718]
 [   1  895]]


C:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [28]:
model = LogisticRegression( class_weight='balanced')
model.fit(X, y)
expected = y
predicted = model.predict(X)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

        0.0       1.00      0.25      0.40      8959
        1.0       0.12      1.00      0.21       896

avg / total       0.92      0.32      0.38      9855

[[2241 6718]
 [   1  895]]


C:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [29]:
model = LogisticRegression(penalty='f1')
model.fit(X, y)
expected = y
predicted = model.predict(X)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

C:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Unsupported set of arguments: The combination of penalty='f1' and loss='logistic_regression' is not supported, Parameters: penalty='f1', loss='logistic_regression', dual=False

In [30]:
model = LogisticRegression(penalty='l1')
model.fit(X, y)
expected = y
predicted = model.predict(X)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

        0.0       0.91      1.00      0.95      8959
        1.0       1.00      0.03      0.07       896

avg / total       0.92      0.91      0.87      9855

[[8959    0]
 [ 865   31]]


C:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
from sklearn import svm
model = svm.SVC()
expected = y
predicted = model.predict(X)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

NotFittedError: This SVC instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [32]:
model.fit(X, y)
expected = y
predicted = model.predict(X)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

        0.0       0.91      1.00      0.95      8959
        1.0       1.00      0.03      0.05       896

avg / total       0.92      0.91      0.87      9855

[[8959    0]
 [ 873   23]]


C:\Anaconda2\lib\site-packages\sklearn\svm\base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


In [33]:
model = svm.SVC(class_weight="balanced")
model.fit(X, y)
expected = y
predicted = model.predict(X)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

        0.0       0.99      0.26      0.41      8959
        1.0       0.12      0.98      0.21       896

avg / total       0.91      0.32      0.39      9855

[[2318 6641]
 [  16  880]]


C:\Anaconda2\lib\site-packages\sklearn\svm\base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


In [34]:
fpr, tpr, thresholds = metrics.roc_curve(expected, predicted, pos_label=2)
metrics.auc(fpr, tpr)

C:\Anaconda2\lib\site-packages\sklearn\metrics\ranking.py:534: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


nan

In [35]:
expected

event
city             date             
Rio de Janeiro   2014-01-01      1
                 2014-01-02      1
                 2014-01-03      1
                 2014-01-04      1
                 2014-01-05      1
                 2014-01-06      1
                 2014-01-07      1
                 2014-01-08      1
                 2014-01-09      1
                 2014-01-10      1
                 2014-01-11      1
                 2014-01-12      1
                 2014-01-13      1
                 2014-01-14      1
                 2014-01-15      1
                 2014-01-16      1
                 2014-01-17      1
                 2014-01-18      1
                 2014-01-19      0
                 2014-01-20      1
                 2014-01-21      1
                 2014-01-22      1
                 2014-01-23      1
                 2014-01-24      1
                 2014-01-25      0
                 2014-01-26      0
                 2014-01-27      1
                 2014-01-28      1
                 2014-01-29      1
                 2014-01-30      1
...                            ...
Distrito Federal 2014-12-02      0
                 2014-12-03      0
                 2014-12-04      0
                 2014-12-05      0
                 2014-12-06      0
                 2014-12-07      0
                 2014-12-08      0
                 2014-12-09      0
                 2014-12-10      0
                 2014-12-11      0
                 2014-12-12      0
                 2014-12-13      0
                 2014-12-14      0
                 2014-12-15      0
                 2014-12-16      0
                 2014-12-17      0
                 2014-12-18      0
                 2014-12-19      0
                 2014-12-20      0
                 2014-12-21      0
                 2014-12-22      0
                 2014-12-23      0
                 2014-12-24      0
                 2014-12-25      0
                 2014-12-26      0
                 2014-12-27      0
                 2014-12-28      0
                 2014-12-29      0
                 2014-12-30      0
                 2014-12-31      0

[9855 rows x 1 columns]

In [36]:
expected.describe

<bound method DataFrame.describe of                              event
city             date             
Rio de Janeiro   2014-01-01      1
                 2014-01-02      1
                 2014-01-03      1
                 2014-01-04      1
                 2014-01-05      1
                 2014-01-06      1
                 2014-01-07      1
                 2014-01-08      1
                 2014-01-09      1
                 2014-01-10      1
                 2014-01-11      1
                 2014-01-12      1
                 2014-01-13      1
                 2014-01-14      1
                 2014-01-15      1
                 2014-01-16      1
                 2014-01-17      1
                 2014-01-18      1
                 2014-01-19      0
                 2014-01-20      1
                 2014-01-21      1
                 2014-01-22      1
                 2014-01-23      1
                 2014-01-24      1
                 2014-01-25      0
                 20

In [37]:
expected.event.describe

<bound method Series.describe of city              date      
Rio de Janeiro    2014-01-01    1
                  2014-01-02    1
                  2014-01-03    1
                  2014-01-04    1
                  2014-01-05    1
                  2014-01-06    1
                  2014-01-07    1
                  2014-01-08    1
                  2014-01-09    1
                  2014-01-10    1
                  2014-01-11    1
                  2014-01-12    1
                  2014-01-13    1
                  2014-01-14    1
                  2014-01-15    1
                  2014-01-16    1
                  2014-01-17    1
                  2014-01-18    1
                  2014-01-19    0
                  2014-01-20    1
                  2014-01-21    1
                  2014-01-22    1
                  2014-01-23    1
                  2014-01-24    1
                  2014-01-25    0
                  2014-01-26    0
                  2014-01-27    1
                  20

In [38]:
fpr, tpr, thresholds = metrics.roc_curve(expected, predicted, pos_label=2)
metrics.auc(fpr, tpr)

nan

In [39]:
fpr, tpr, thresholds = metrics.roc_curve(expected, predicted, pos_label=1)
metrics.auc(fpr, tpr)

0.62043854543715016

In [40]:
models = [LogisticRegression(penalty='l1', class_weight='balanced'),
          SVC(class_weight="balanced"),
          ]

for m in models:
    m.fit(X, y)
    expected = y
    predicted = m.predict(X)
    print(metrics.classification_report(expected, predicted))
    print(metrics.confusion_matrix(expected, predicted))
    fpr, tpr, thresholds = metrics.roc_curve(expected, predicted, pos_label=1)
    print "AOC: ", str(metrics.auc(fpr, tpr))

NameError: name 'SVC' is not defined

In [41]:
from sklearn.svm import SVC


In [42]:
models = [LogisticRegression(penalty='l1', class_weight='balanced'),
          SVC(class_weight="balanced"),
          ]

for m in models:
    m.fit(X, y)
    expected = y
    predicted = m.predict(X)
    print(metrics.classification_report(expected, predicted))
    print(metrics.confusion_matrix(expected, predicted))
    fpr, tpr, thresholds = metrics.roc_curve(expected, predicted, pos_label=1)
    print "AOC: ", str(metrics.auc(fpr, tpr))

C:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Anaconda2\lib\site-packages\sklearn\svm\base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


             precision    recall  f1-score   support

        0.0       1.00      0.26      0.42      8959
        1.0       0.12      1.00      0.21       896

avg / total       0.92      0.33      0.40      9855

[[2351 6608]
 [   0  896]]
AOC:  0.631208840272
             precision    recall  f1-score   support

        0.0       0.99      0.26      0.41      8959
        1.0       0.12      0.98      0.21       896

avg / total       0.91      0.32      0.39      9855

[[2318 6641]
 [  16  880]]
AOC:  0.620438545437


In [43]:
m.fit(X, y)

C:\Anaconda2\lib\site-packages\sklearn\svm\base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [44]:
m.fit(X, y.event)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [45]:
le = LabelEncoder()

NameError: name 'LabelEncoder' is not defined

In [46]:
from ..preprocessing import LabelEncoder

ValueError: Attempted relative import in non-package

In [47]:
from sklearn import preprocessing

In [48]:
from preprocessing import LabelEncoder

ImportError: No module named preprocessing

In [49]:
le = LabelEncoder()

NameError: name 'LabelEncoder' is not defined

In [50]:
le = preprocessing.LabelEncoder()